## Теперь о монетизации - Sleepy работает по подписке: 990 рублей в месяц. Первые две недели бесплатно, затем клиент видит пейвол (от «pay wall» - как бы стена, которая тебя не пускает дальше, пока ты не оплатишь) и, если привязывает карту, продолжает пользоваться подпиской, а деньги списываются автоматически.

## Если говорить о привлечении, то это молодой венчурный стартап, поэтому привлечение одного пользователя (даже если он не заплатит) довольно дорогое - 500 рублей.

## Данных у вас немного - вам дан датасет всего из двух полей:


## user - номер пользователя в системе

## dt - дата его захода в приложение

## Данные устроены таким образом, что фиксируется только один заход пользователя за день - одинаковых пар "пользователь - дата" быть не может. 
## Кроме того, фиксируются только заходы пользователей на триальной версии и тех, кто прошел пейвол. 
## Если человек открыл приложение после 14 дня и не привязал карту - такой заход не фиксируется.

In [1]:
import pandas as pd
import math
import matplotlib.pyplot as plt

In [2]:
# n day retention


# WIN
users_sleepy = pd.read_csv(r'C:\Users\Incognitus\Downloads\entries.csv', sep=';') 
# MAC
# users_sleepy = pd.read_csv(r'/Users/vladislavlipkin/Downloads/entries.csv', sep=';') 

In [3]:
users_sleepy.head(5)

,user,dt,Unnamed: 2
0,0,12.11.2023,NaN
1,0,13.11.2023,NaN
2,0,14.11.2023,NaN
3,0,16.11.2023,NaN
4,0,17.11.2023,NaN


In [4]:
# --- подготовка данных ---
users_filter = users_sleepy.loc[:, ['user', 'dt']]
users_filter['dt'] = pd.to_datetime(users_filter['dt'], format='%d.%m.%Y')
users_filter['month_start'] = users_filter['dt'].dt.to_period('M').dt.to_timestamp()
users_filter['dt_reg'] = users_filter.groupby('user')['dt'].transform('min')
users_filter['diff'] = (users_filter['dt'] - users_filter['dt_reg']).dt.days
users_filter['week'] = round(users_filter['diff'] / 7, 0)
users_filter['month'] = round(users_filter['diff'] / 30, 0)


# --- считаем общее количество пользователей ---
all_users = users_filter['user'].nunique()

# --- retention по разным периодам ---
users_filter['N_DAY_retention']   = users_filter.groupby('diff')['user'].transform('nunique') / all_users * 100
users_filter['N_WEEK_retention']  = users_filter.groupby('week')['user'].transform('nunique') / all_users * 100
users_filter['N_MONTH_retention'] = users_filter.groupby('month')['user'].transform('nunique') / all_users * 100


# 🔹🔹🔹 Универсальная переменная для переключения периода 🔹🔹🔹
period = 'day'     # варианты: 'day', 'week', 'month'


# --- выбираем колонку и подписи в зависимости от периода ---
if period == 'day':
    idx_col = 'diff'
    period_name = 'Day'
elif period == 'week':
    idx_col = 'week'
    period_name = 'Week'
elif period == 'month':
    idx_col = 'month'
    period_name = 'Month'
else:
    raise ValueError("period must be 'day', 'week', or 'month'")


# --- создаем универсальный pivot ---
pivot = (
    users_filter
    .groupby(idx_col, as_index=False)['user']
    .nunique()
    .rename(columns={'user': 'unique_users'})
)

pivot['retention'] = pivot['unique_users'] / all_users * 100
pivot['period'] = period_name  # добавляем столбец для понятного вывода

# --- создаем полный диапазон индексов ---
full_range = pd.DataFrame({idx_col: range(int(pivot[idx_col].min()), int(pivot[idx_col].max()) + 1)})
pivot_full = full_range.merge(pivot, on=idx_col, how='left')

# --- вывод ---
print(f"Retention by {period_name}")
display(pivot_full.head(20))
print(pivot_full['retention'].sum() / 100)

Retention by Day


,diff,unique_users,retention,period
0,0,2000.0,100.00,Day
1,1,1039.0,51.95,Day
2,2,1052.0,52.60,Day
3,3,1055.0,52.75,Day
4,4,1024.0,51.20,Day
5,5,968.0,48.40,Day
6,6,984.0,49.20,Day
7,7,927.0,46.35,Day
8,8,954.0,47.70,Day
9,9,958.0,47.90,Day


26.162500000000005


In [5]:
# 25 minutes
users_filter.sort_values(by='dt', ascending=True).head(20)

,user,dt,month_start,dt_reg,diff,week,month,N_DAY_retention,N_WEEK_retention,N_MONTH_retention
24607,940,2023-02-18,2023-02-01,2023-02-18,0,0.0,0.0,100.00,100.0,100.0
32045,1224,2023-02-18,2023-02-01,2023-02-18,0,0.0,0.0,100.00,100.0,100.0
37567,1438,2023-02-18,2023-02-01,2023-02-18,0,0.0,0.0,100.00,100.0,100.0
51147,1960,2023-02-18,2023-02-01,2023-02-18,0,0.0,0.0,100.00,100.0,100.0
19446,740,2023-02-19,2023-02-01,2023-02-19,0,0.0,0.0,100.00,100.0,100.0
14570,553,2023-02-19,2023-02-01,2023-02-19,0,0.0,0.0,100.00,100.0,100.0
50719,1941,2023-02-19,2023-02-01,2023-02-19,0,0.0,0.0,100.00,100.0,100.0
17388,659,2023-02-19,2023-02-01,2023-02-19,0,0.0,0.0,100.00,100.0,100.0
51148,1960,2023-02-19,2023-02-01,2023-02-18,1,0.0,0.0,51.95,100.0,100.0
32046,1224,2023-02-19,2023-02-01,2023-02-18,1,0.0,0.0,51.95,100.0,100.0


In [6]:
# среднее MAU за весь период
user_in_month = users_filter.groupby('month_start')['user'].nunique().reset_index(name='unique_users')
average_mau = user_in_month['unique_users'].mean()
average_mau

458.9655172413793

In [8]:
# среднее DAU за весь период

# --- считаем количество уникальных пользователей по каждой дате ---
user_in_day = (
    users_filter.groupby('dt')['user']
    .nunique()
    .reset_index(name='unique_users')
)
# --- создаем полный список дат ---

full_dates = pd.DataFrame({
    'dt': pd.date_range(users_filter['dt'].min(), users_filter['dt'].max())
})
# --- добавляем пропущенные даты ---
user_in_day_full = (
    full_dates
    .merge(user_in_day, on='dt', how='left')
    .fillna({'unique_users': 0})  # ставим 0, если в этот день нет пользователей
)
# --- считаем среднее DAU ---
average_dau = user_in_day_full['unique_users'].mean()
average_dau

60.984848484848484

In [9]:
# Sticky_factor
print(f'average_DAU: {round(average_dau, 1)}')
print(f'average_MAU: {round(average_mau, 1)}')
print()
print(f'Sticky_factor: {average_dau / average_mau * 100}')

average_DAU: 61.0
average_MAU: 459.0

Sticky_factor: 13.287457596247979


In [ ]:

fig, ax1 = plt.subplots(figsize=(9, 5))

# --- Первая ось: Retention ---
color = 'tab:blue'
ax1.set_xlabel(f'{period_name} after registration')
ax1.set_ylabel('Retention, %', color=color)
ax1.plot(pivot_full[idx_col], pivot_full['retention'], color=color, marker='o', linewidth=2, label='Retention %')
ax1.tick_params(axis='y', labelcolor=color)
ax1.grid(True, which='major', axis='both', linestyle='--', alpha=0.5)

# --- Вторая ось: количество уникальных пользователей ---
ax2 = ax1.twinx()  # создаём вторую ось Y
color = 'tab:orange'
ax2.set_ylabel('Unique users', color=color)
ax2.bar(pivot_full[idx_col], pivot_full['unique_users'], color=color, alpha=0.3, label='Unique users')
ax2.tick_params(axis='y', labelcolor=color)

# --- Заголовок и оформление ---
plt.title(f'Retention and Unique Users by {period_name}')
fig.tight_layout()
plt.show()

In [ ]:
6 * 990

In [ ]:
users_filter

In [ ]:
# users_filter['month'] = round(users_filter['diff'] / 30, 0)
# users_filter['week'] = round(users_filter['diff'] / 7, 0)
users_filter

In [ ]:
print(f'count of month: {users_filter['month'].sum() / 100}')
print(f'count of week: {users_filter['week'].sum() / 100}')

In [ ]:
# count users paying
cpu = users_filter.query('diff >= 14')['user'].nunique()
print(f'количество людей купивших подписку : {cpu}')
print()
price_all_users = all_users * 500
net_profit = cpu * 990 - price_all_users
print(f'стоимость привлечения за всех пользователей {price_all_users}')
print()
print(f'выручка за все проданные подписки: {cpu * 990}')
print()
print(f'чистая прибыль с учетом затратов: {net_profit}')

In [ ]:
(full_range / users_filter['user'].nunique())['diff'].sum() / 30

In [ ]:
pivot_full['retention'].sum() / 100

In [ ]:
uniq_users = users_filter.query('14 <= diff <= 16')['user'].nunique()
print(f'количество уникальных пользователей: {uniq_users}')
print(f'выручка: {uniq_users * 990}')

In [ ]:
pivot_full.query('diff >= 14').sort_values('retention', ascending=False)

In [ ]:
users_filter.query('diff >= 14')['user'].nunique() - 500

In [ ]:
pivot_full.query('diff >= 14')